In [3]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
import os

In [4]:
# Setup
IMAGE_SIZE = (128, 128)
BATCH_SIZE = 32
EPOCHS = 10

In [5]:
# Load base model
base_model = MobileNetV2(include_top=False, weights="imagenet", input_shape=(128, 128, 3))
base_model.trainable = False  # freeze layers

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 5s 1us/step


In [6]:
# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation="sigmoid")(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [7]:

# Compile
model.compile(optimizer=Adam(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])


In [6]:
# Compile
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [8]:
# Data generators
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_gen = train_datagen.flow_from_directory(
    "chest_xray/train",  # path to your dataset
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_gen = train_datagen.flow_from_directory(
    "chest_xray/train",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

Found 4173 images belonging to 2 classes.
Found 1043 images belonging to 2 classes.


In [9]:
# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [11]:
# Train
history = model.fit(
    train_gen,
    epochs=EPOCHS,
    validation_data=val_gen,
    callbacks=[early_stop]
)

C:\Users\INFINITE COMPUTER\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 124s 893ms/step - accuracy: 0.6311 - loss: 0.7301 - val_accuracy: 0.7987 - val_loss: 0.3900
Epoch 2/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 111s 846ms/step - accuracy: 0.7691 - loss: 0.4851 - val_accuracy: 0.9003 - val_loss: 0.2896
Epoch 3/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 111s 847ms/step - accuracy: 0.8299 - loss: 0.3583 - val_accuracy: 0.9243 - val_loss: 0.2425
Epoch 4/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 110s 841ms/step - accuracy: 0.8479 - loss: 0.3271 - val_accuracy: 0.9300 - val_loss: 0.2152
Epoch 5/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 110s 838ms/step - accuracy: 0.8794 - loss: 0.2934 - val_accuracy: 0.9367 - val_loss: 0.1953
Epoch 6/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 110s 840ms/step - accuracy: 0.8868 - loss: 0.2631 - val_accuracy: 0.9338 - val_loss: 0.1850
Epoch 7/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 110s 843ms/step - accuracy: 0.9000 - loss: 0.2465 - val_accuracy: 0.9386 - val_loss: 0.1741
Epoch 8/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 110s 844ms/step - accuracy: 0.8950 -

In [27]:
model.save("model.keras", save_format="keras")